In [1]:
!pip install onnx
!pip install onnxruntime
!pip install onnx2pytorch
!pip install maraboupy
#!pip install tensorflow[and-cuda]==2.13.0

In [ ]:
# import os
# os.kill(os.getpid(), 9)

In [2]:
import onnx
from onnx2pytorch import ConvertModel
# iris_onnx_path =  'iris_model_60.onnx'  # Adjust file extension if needed
# PATTERN_PATH = 'relu_states'

from google.colab import drive
drive.mount('/content/drive')

folder_path = '/content/drive/My Drive/oopsla_nap'
iris_onnx_path = folder_path + '/iris_model_60.onnx'  # Adjust file extension if needed
PATTERN_PATH = folder_path + '/relu_states_allen.txt'  # Adjust file extension if needed

# Adjust file extension if needed
# onnx_model = onnx.load(iris_onnx_path)
# model = ConvertModel(onnx_model)
# data=iris.data
# data
# X_train
#labels=iris.target #target_names': array(['setosa', 'versicolor', 'virginica'], dtype='<U10' - sepal length in cm\n        - sepal width in cm\n        - petal length in cm\n        - petal width in cm\n

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from maraboupy import Marabou, MarabouCore, MarabouUtils
import json
from typing import Tuple, List
import logging
import pandas

def init_network():
  network = Marabou.read_onnx(iris_onnx_path)
  return network

EPSILON = 0.5
MAX_TIME = 300 #in seconds
M_OPTIONS: MarabouCore.Options = Marabou.createOptions(verbosity=0, numWorkers=10, timeoutInSeconds=MAX_TIME)

def convert_keys_to_int(x):
    if isinstance(x, dict):
        return {int(k) if k.isdigit() else k: convert_keys_to_int(v) for k, v in x.items()}
    return x

def parse_raw_idx(raw_idx: int) -> Tuple[int, int, int]:
    n_relus = 20
    offset = 7
    layer = raw_idx // n_relus
    idx = raw_idx % n_relus
    marabou_idx = 2*n_relus*layer + idx + offset
    return layer, idx, marabou_idx

with open(PATTERN_PATH, "r") as f:
  STABLE_PATTERNS = json.load(f)
  STABLE_PATTERNS=convert_keys_to_int(STABLE_PATTERNS)
loc = 0.5
radus = 0.5
non_restricted_dim = []

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
for k,v in STABLE_PATTERNS.items():
  print(f'label={k}')
  # for neuron,values in v.items():
  #   print(f"neuron={neuron}")
network=init_network()
len(network.equList)

label=0
label=1
label=2


63

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


# Load the Iris dataset
data = load_iris()
X, y = data.data, data.target

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Define the neural network architecture
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 20)  # Input layer to hidden layer
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(20, 20) # Hidden layer to hidden layer
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(20, 20) # Hidden layer to hidden layer
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(20, 3)  # Hidden layer to output layer

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x

# Initialize the network
model = IrisNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Print loss every 10 epochs
    if epoch % 10 == 9:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Check if the model trains without error up to this point
# "Model trained successfully!"


Epoch 10, Loss: 0.8084816932678223
Epoch 20, Loss: 0.200725257396698
Epoch 30, Loss: 0.18347160518169403
Epoch 40, Loss: 0.02535819262266159
Epoch 50, Loss: 0.3132612109184265
Epoch 60, Loss: 0.020115939900279045
Epoch 70, Loss: 0.057707611471414566
Epoch 80, Loss: 0.0010892553254961967
Epoch 90, Loss: 0.0037069518584758043
Epoch 100, Loss: 0.0022463109344244003


In [6]:
import onnx
from onnx2pytorch import ConvertModel


# onnx_model = onnx.load("iris_model.onnx")
# model = ConvertModel(onnx_model)

activations = {}

def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

i=1
# Assuming the ReLU layers are named similarly to your PyTorch model, you can add hooks like this
for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.ReLU):
        # Register the hook
        layer.register_forward_hook(get_activation(f'relu{i}'))
        i=i+1

# Now, when you run a forward pass, the hooks will store the ReLU activations
with torch.no_grad():
    output = model(X_train_tensor[0].unsqueeze(0))

# Check the captured activations
activations
# Remove hooks (to clean up)
# hook1.remove()
# hook2.remove()

{'relu1': tensor([[0.0000, 0.3371, 2.7642, 1.3335, 0.0000, 0.2346, 0.0000, 2.2037, 0.0000,
          0.0000, 0.0000, 0.0000, 1.3703, 0.0000, 1.2240, 1.4983, 1.6326, 0.0000,
          0.0000, 0.8695]]),
 'relu2': tensor([[0.0000, 3.4158, 3.2105, 2.9432, 0.0000, 0.0000, 0.0000, 3.4038, 2.6163,
          3.2309, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.1544, 0.0000, 0.0899,
          2.1026, 3.6832]]),
 'relu3': tensor([[0.0000, 4.3137, 6.9853, 0.0000, 0.5758, 0.0000, 0.0000, 0.2477, 0.0000,
          3.1130, 5.6233, 2.3839, 0.0000, 4.6847, 0.8657, 5.9987, 7.1666, 0.0000,
          0.0000, 0.0000]])}

In [ ]:
#accuracy:
i=0
with torch.no_grad():
  for inputs,label in zip(X_test_tensor, y_test_tensor):
          outputs = model(inputs.unsqueeze(0))
          #print(outputs)
          pred=outputs.argmax() #output not normalized
          if pred==label:
            i=i+1
print(f'{i}/{len(X_test_tensor)}')

model(X_train_tensor[0].unsqueeze(0))

onnx_file_path = "iris_model_60.onnx"
torch.onnx.export(model, torch.randn(1, 4), onnx_file_path, verbose=False)

30/30


find states:

In [ ]:
states={}
for i in range(3):
  states[i]={}

with torch.no_grad():
  for neuron in range(60):
    for idx, (inputs, labels) in enumerate(train_loader):
      for i,t in zip(inputs,labels):
        t=t.item()
        outputs = model(i.unsqueeze(0))
        out=activations[f'relu{(neuron//20)+1}'][:,neuron%20].tolist()[0]
        if not neuron in states[t]:
          states[t][neuron]=[]
        states[t][neuron].append(out)

with open("relu_states", "w") as fp:
  json.dump(states, fp)

states[0]

# states
# json
# label:{{neuron: states}, }

{0: [0.828767716884613,
  0.6903653144836426,
  0.6370339393615723,
  1.246646523475647,
  0.8410075306892395,
  1.0450611114501953,
  1.4888828992843628,
  0.0,
  0.7377094626426697,
  0.7998632788658142,
  1.1056742668151855,
  1.649401307106018,
  0.8751844167709351,
  0.7333645820617676,
  1.2010773420333862,
  1.5152419805526733,
  0.9550075531005859,
  1.0494059324264526,
  0.9167447090148926,
  0.5503205060958862,
  0.6676362156867981,
  1.1625556945800781,
  0.750876784324646,
  0.5013340711593628,
  1.1982731819152832,
  0.6580435037612915,
  0.7823510766029358,
  0.6723171472549438,
  0.8734093308448792,
  1.2464121580123901,
  0.6151769161224365,
  0.40757331252098083,
  1.0747358798980713,
  0.9137061238288879,
  0.9532325267791748,
  1.0205014944076538,
  1.2062480449676514,
  0.45926231145858765,
  0.8200780153274536,
  0.9129357933998108],
 1: [1.266777515411377,
  1.1708238124847412,
  1.143419861793518,
  1.1619681119918823,
  1.940536379814148,
  1.3608477115631104,
 

###### initialize marabou

In [ ]:
#start with all relu，删减直到不能verify？

In [ ]:
# network=init_network()
# network.outputVars #4,5,6
# network.inputVars #0,1,2,3
# network.reluList #(i+10) for i in range(7,37)
# network.numVars #47
# offset=7

In [ ]:
STABLE_PATTERNS[0]

In [9]:
# network = init_network()
# label=2
# other_label=1
# label_states=STABLE_PATTERNS[label]
# network = add_relu_constraints(network, label_states) #previously commented out??
# offset = network.outputVars[0][0][0]
# for i in range(4):
#   network.setLowerBound(i, 0)
#   network.setUpperBound(i, 1)

# for neuron,values in label_states.items():
#         layer, idx, marabou_idx = parse_raw_idx(neuron)
#         #print(f'{neuron}: {values}')
#         if values.count(0) == len(values): #len(X_train)*0.5:
#             constraint = MarabouUtils.Equation(MarabouCore.Equation.LE)
#             constraint.addAddend(1, marabou_idx)
#             constraint.setScalar(-0.001)
#         else:
#             constraint = MarabouUtils.Equation(MarabouCore.Equation.GE)
#             constraint.addAddend(1, marabou_idx)
#             constraint.setScalar(0.001)
#         network.addEquation(constraint)
#         # import pdb;pdb.set_trace()

# #add output constraint
# constraint = MarabouUtils.Equation(MarabouCore.Equation.GE)
# constraint.addAddend(1, other_label+offset)
# constraint.addAddend(-1, label+offset)
# constraint.setScalar(0.001)
# network.addEquation(constraint)


# #add additional bounds here
# exit_code: str
# exit_code, vals, stats = network.solve(options=M_OPTIONS)
# len(network.equList)

unsat


184

StatRefine:

In [ ]:
from math import e
#a=[0,1,0,1,1,…] = some binary nap set
#theorem based on s: p(F(a) = 0) ~= (d/(d+1))^d = e^(-1) as d -> inf.
#want to optimize: p(F(a) = 0) ~= e^(−1) ~= t, gradient descend on alpha.
#Intuition: alpha高, contains 更多 relus, p(F(a))更高. 所以F(a)是1的时候，descend theta, alpha变小

all_relus=60   # |J|
n=10 # num samples
theta=5
step_size=0.1
target_prob=e**(-1)

def get_alpha(theta):
  return (1+e**(-theta))**(-1)

def F(a):
  exit_code, running_time = check_pattern(STABLE_PATTERNS[label],
                                          label=int(label), other_label = other_label, a=a)
  return exit_code #0 means robust, good!


def sample(alpha, a_L, all_relus):
  a=a_L
  for j in range(all_relus):
    if a_L[j]==0:
      a[j]= np.random.choice([0, 1], size=(n, 1), p=[1-alpha, alpha]).reshape(n,) #turn on with probability alpha
  return a

def StatRefine(a_L):
	if F(a_L)=="unsat": #or current set(a_L)size太大了及时止损 but adjust alpha now, no need to care for s. In practice, s is unknown, so we seek a mechanism for setting α without this knowledge
    return a_L
  examples=[]
  alpha=get_alpha()
	for i in range(n): # generate example
		a_i= sample(alpha, a_L, all_relu)
    if F(a_i) == "unsat":
       F_a_i = 0
    else:
       F_a_i = 1
    theta=theta-step_size*(1-F_a_i-target_prob) #If |marabou(a_i)=unsat| is large, decrease alpha (gradient descend??)
    examples.append((a_i,F_a_i))

  n=[]
	for j in range(all_relus):
		if j not in a_L:
			for (a_i, F_a_i) in examples:
				if a_i[j]==1 and F_a_i=="unsat":
					n[j]+=1

	j_max=n.argmax()
return StatRefine(a_L[j]=1)

#run StatRefine
a=[0 in range (all_relus)]
StatRefine(a)

In [5]:
def add_relu_constraints(network: Marabou.MarabouNetworkNNet, label_states, a=None)->Marabou.MarabouNetworkNNet:
    """
    Add stable relus constraints to the Marabou network
    """
    if a:
      for neuron,values in label_states.items(): #everything
        if a[neuron]==1: #include this relu
          layer, idx, marabou_idx = parse_raw_idx(neuron)
          #print(f'{neuron}: {values}')
          if values.count(0) > len(values)*0.5: #len(X_train)*0.5:  len(values)
              constraint = MarabouUtils.Equation(MarabouCore.Equation.LE)
              constraint.addAddend(1, marabou_idx)
              constraint.setScalar(-0.001)
          else:
              constraint = MarabouUtils.Equation(MarabouCore.Equation.GE)
              constraint.addAddend(1, marabou_idx)
              constraint.setScalar(0.001)
          network.addEquation(constraint)

    else:
      for neuron,values in label_states.items():
          layer, idx, marabou_idx = parse_raw_idx(neuron)
          #print(f'{neuron}: {values}')
          if values.count(0) > len(values)*0.5: #len(X_train)*0.5:  len(values)
              constraint = MarabouUtils.Equation(MarabouCore.Equation.LE)
              constraint.addAddend(1, marabou_idx)
              constraint.setScalar(-0.001)
          else:
              constraint = MarabouUtils.Equation(MarabouCore.Equation.GE)
              constraint.addAddend(1, marabou_idx)
              constraint.setScalar(0.001)
          network.addEquation(constraint)
        # import pdb;pdb.set_trace()

    return network



def check_pattern(label_states, label: int, other_label: int, a=None)->Tuple[str, int]: #relu_check_list: List[int], relu_val: List[int]
    """
    In ACAS, the prediction is the label with smallest value.
    So we check that label - other_label < 0 forall input
    by finding assignments for label - other_label >=0
    """
    print("--------CHECK PATTERN: output_{} is always less than output_{} ? --------".format(label, other_label))
    network = init_network()
    network = add_relu_constraints(network, label_states,a) #previously commented out??
    offset = network.outputVars[0][0][0]
    for i in range(4):
      network.setLowerBound(i, 0)
      network.setUpperBound(i, 1)
    #add output constraint
    constraint = MarabouUtils.Equation(MarabouCore.Equation.GE)
    constraint.addAddend(1, other_label+offset)
    constraint.addAddend(-1, label+offset)
    constraint.setScalar(0.001)
    network.addEquation(constraint)


    #add additional bounds here
    exit_code: str
    import pdb;pdb.set_trace()
    exit_code, vals, stats = network.solve(options=M_OPTIONS)

    running_time:int = stats.getTotalTimeInMicro()

    return exit_code, running_time

def main():
    res = [[-1.]*3 for i in range(3)]
    # print(res)
    for label in range(3):
        print(f"For label {label}, check if its stable RELU pattern guarantees the output")
        for other_label in range(3):#range(10):
            if other_label == int(label):
                continue
            # relu_check_list = STABLE_PATTERNS[label]["stable_idx"]
            # relu_val = STABLE_PATTERNS[label]["val"]
            exit_code, running_time = check_pattern(STABLE_PATTERNS[label], label=int(label), other_label = other_label)
            if exit_code=="sat":
                res[int(label)][other_label] = "SAT:{}".format(running_time/10**6)
                break
            elif exit_code=="unsat":
                res[int(label)][other_label] = "UNS:{}".format(running_time/10**6)

            else:
                res[int(label)][other_label] = exit_code

    res = pandas.DataFrame(res)
    print(res)

#main()


In [ ]:
main()

In [ ]:
# for label in [2]:
#   relu_label=STABLE_PATTERNS[label].copy()
#   print(type(relu_label))
#   def func(relus):
#     for other_label in range(3):
#       exit_code, running_time = check_pattern(relus, label=int(label), other_label = other_label)
#       if exit_code=="sat":
#         return False
#     return True

#   flag=func(relu_label)
#   while flag:
#     popped_key=sorted(relu_label.keys())[-1]
#     relu_label.pop(popped_key)
#     print(f"popped neuron={popped_key}")
#     print(type(relu_label))
#     flag=func(relu_label)
#   print(f"for label {label}: remaining neurons= {relu_label.keys()}")


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 336, in set_trace
    sys.settrace(self.trace_dispatch)



<class 'dict'>
--------CHECK PATTERN: output_2 is always less than output_0 ? --------
> <ipython-input-5-b0fb89cf5aa7>(47)check_pattern()
     45     exit_code: str
     46     import pdb;pdb.set_trace()
---> 47     exit_code, vals, stats = network.solve(options=M_OPTIONS)
     48 
     49     running_time:int = stats.getTotalTimeInMicro()

ipdb> continue



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/bdb.py", line 347, in set_continue
    sys.settrace(None)



In [ ]:
# def func(relus):
#     for other_label in range(3):
#       exit_code, running_time = check_pattern(relus, label=int(label), other_label = other_label)
#       if exit_code=="sat":
#         return False
#     return True

# def scancoarsen(relu_label):
#     keyss = relu_label.keys()
#     nap = []
#     relu_label_copy = relu_label.copy()
#     for k in keyss:
#         relu_label_copy.pop(k)
#         if func(relu_label_copy):
#             relu_label.pop(k)
#             continue
#         else:
#             nap.append(k)
#             relu_label_copy = relu_label.copy()
#     return relu_label.keys()

# for label in range(3):
#     relu_label=STABLE_PATTERNS[label].copy()
#     scancoarsen(relu_label)

--------CHECK PATTERN: output_0 is always less than output_0 ? --------
unsat
--------CHECK PATTERN: output_0 is always less than output_1 ? --------
unsat
--------CHECK PATTERN: output_0 is always less than output_2 ? --------
unsat


RuntimeError: dictionary changed size during iteration

scanCoarsen

In [ ]:
def func(relus):
    for other_label in range(3):
        exit_code, running_time = check_pattern(relus, label=int(label), other_label=other_label)
        if exit_code == "sat":
            return False
    return True

def scancoarsen(relu_label):
    keys = list(relu_label.keys())  # Convert keys to a list to avoid runtime error while modifying the dictionary
    for k in keys:
        print(f"consider neuron={k}")
        vals = relu_label[k]
        relu_label_copy = relu_label.copy()
        relu_label_copy.pop(k)
        if func(relu_label_copy):
            relu_label.pop(k)
            print(relu_label.keys())
            continue
        else:
            relu_label[k] = vals
    return list(relu_label.keys())  # Return the remaining keys

# # Assuming STABLE_PATTERNS is a list of dictionaries
# for label in range(1):
#     relu_label = STABLE_PATTERNS[label].copy()
#     remaining_keys = scancoarsen(relu_label)
#     print(f"Remaining keys after coarsening for label {label}: {remaining_keys}")


label = 2
relu_label = STABLE_PATTERNS[label].copy()
remaining_keys = scancoarsen(relu_label)
print(f"Remaining keys after coarsening for label {label}: {remaining_keys}")

consider neuron=0
--------CHECK PATTERN: output_2 is always less than output_0 ? --------
> <ipython-input-8-b0fb89cf5aa7>(47)check_pattern()
     45     exit_code: str
     46     import pdb;pdb.set_trace()
---> 47     exit_code, vals, stats = network.solve(options=M_OPTIONS)
     48 
     49     running_time:int = stats.getTotalTimeInMicro()

ipdb> continue


#### relu_label = STABLE_PATTERNS[label].copy()
type(relu_label)

In [ ]:
len(relu_label)

58

In [ ]:
import numpy as np

def generate_samples(n, k, alpha, j):
    # Calculate the number of samples with the jth element equal to 1
    num_ones = int(alpha * n)

    # Generate n samples with the specified probability alpha
    samples = np.random.choice([0, 1], size=(n, k), p=[0.5, 0.5])
    print(samples)
    j_col = np.random.choice([0, 1], size=(n, 1), p=[1-alpha, alpha]).reshape(n,)
    print(j_col)
    samples[:,j] = j_col

    # Set the jth element to 1 in the required number of samples


    return samples

# Example usage:
n = 10  # Number of samples
k = 4   # Length of the list
alpha = 0.1  # Probability
j = 1   # Index of the element to be set to 1

generated_samples = generate_samples(n, k, alpha, j)
generated_samples

[[1 1 1 0]
 [1 0 0 1]
 [0 1 0 0]
 [0 1 1 0]
 [0 0 0 0]
 [1 0 0 1]
 [0 0 1 0]
 [0 0 0 0]
 [0 0 0 1]
 [0 0 1 1]]
[0 1 0 1 0 0 0 0 0 0]


array([[1, 0, 1, 0],
       [1, 1, 0, 1],
       [0, 0, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 0, 0],
       [1, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 1]])

In [ ]:
generated_samples[:2, :]

array([[1, 0, 1, 1],
       [1, 0, 0, 1]])

In [ ]:
relu_label=STABLE_PATTERNS[1].copy()
relu_label

NameError: name 'STABLE_PATTERNS' is not defined



---

